## Importações e constantes

In [ ]:
import csv
# from dotenv import get_key
# import psycopg2
# import pandas as pd
from utils import *

# NAME = get_key('.env', 'PG_NAME')
# USER = get_key('.env', 'PG_USER')
# PASSWORD = get_key('.env', 'PG_PASSWORD')
# HOST = get_key('.env', 'PG_HOST')
# PORT = get_key('.env', 'PG_PORT')

# Caminhos para os arquivos CSV utilizados
PROC_ENERGY_CSV = 'bancos/energia_processada.csv'
WORLD_DATA_CSV = 'bancos/WDI_processado.csv'

## Criação de tabelas no banco

In [ ]:
def create_tables(conn):
	"""Cria todas as tabelas seguindo o modelo relacional."""
	cursor = conn.cursor()
	try:
		cursor.execute("""
			-- This script was generated by the ERD tool in pgAdmin 4.
				-- Please log an issue at https://github.com/pgadmin-org/pgadmin4/issues/new/choose if you find any bugs, including reproduction steps.
				BEGIN;


				CREATE TABLE IF NOT EXISTS public.pais
				(
					codigo character varying(5) NOT NULL,
					nome character varying(50) NOT NULL,
					continente character varying(20) NOT NULL,
					PRIMARY KEY (codigo)
				);

				CREATE TABLE IF NOT EXISTS public.grupo
				(
					id_grupo integer NOT NULL,
					sigla character varying(10) NOT NULL,
					PRIMARY KEY (id_grupo)
				);

				CREATE TABLE IF NOT EXISTS public.pais_grupo
				(
					pais_codigo character varying(5) NOT NULL,
					grupo_id_grupo integer NOT NULL
				);

				CREATE TABLE IF NOT EXISTS public.area
				(
					id integer NOT NULL,
					tipo character varying(10) NOT NULL,
					PRIMARY KEY (id)
				);

				CREATE TABLE IF NOT EXISTS public.distribuicao_territorial
				(
					pais_codigo character varying(5) NOT NULL,
					area_id integer NOT NULL,
					ano integer NOT NULL,
					area_km2 double precision NOT NULL
				);

				CREATE TABLE IF NOT EXISTS public.pib
				(
					ano integer NOT NULL,
					codigo_pais character varying NOT NULL,
					valor_usd double precision NOT NULL,
					PRIMARY KEY (ano, codigo_pais)
				);

				CREATE TABLE IF NOT EXISTS public.demografia
				(
					area_id integer NOT NULL,
					pais_codigo character varying(5) NOT NULL,
					ano integer,
					num_habitantes integer
				);

				CREATE TABLE IF NOT EXISTS public.gas
				(
					id_gas integer NOT NULL,
					nome_gas character varying(100) NOT NULL,
					formula_gas character varying(10),
					PRIMARY KEY (id_gas)
				);

				CREATE TABLE IF NOT EXISTS public.emissao_gas
				(
					gas_id_gas integer NOT NULL,
					pais_codigo character varying(5) NOT NULL,
					ano integer NOT NULL,
					emissao_mtco2 double precision NOT NULL
				);

				CREATE TABLE IF NOT EXISTS public.energia
				(
					id_energia integer NOT NULL,
					tipo character varying(100) NOT NULL,
					PRIMARY KEY (id_energia)
				);

				CREATE TABLE IF NOT EXISTS public.interacao_energetica
				(
					id_interacao integer NOT NULL,
					nome character varying(100) NOT NULL,
					unidade character varying(10) NOT NULL,
					PRIMARY KEY (id_interacao)
				);

				CREATE TABLE IF NOT EXISTS public.relatorio_energetico
				(
					codigo_pais character varying(5) NOT NULL,
					id_interacao integer NOT NULL,
					id_energia integer NOT NULL,
					ano integer NOT NULL,
					valor double precision NOT NULL
				);

				ALTER TABLE IF EXISTS public.pais_grupo
					ADD FOREIGN KEY (pais_codigo)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.pais_grupo
					ADD FOREIGN KEY (grupo_id_grupo)
					REFERENCES public.grupo (id_grupo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.distribuicao_territorial
					ADD FOREIGN KEY (pais_codigo)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.distribuicao_territorial
					ADD FOREIGN KEY (area_id)
					REFERENCES public.area (id) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.pib
					ADD FOREIGN KEY (codigo_pais)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.demografia
					ADD FOREIGN KEY (area_id)
					REFERENCES public.area (id) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.demografia
					ADD FOREIGN KEY (pais_codigo)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.emissao_gas
					ADD FOREIGN KEY (gas_id_gas)
					REFERENCES public.gas (id_gas) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.emissao_gas
					ADD FOREIGN KEY (pais_codigo)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.relatorio_energetico
					ADD FOREIGN KEY (codigo_pais)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.relatorio_energetico
					ADD FOREIGN KEY (id_interacao)
					REFERENCES public.interacao_energetica (id_interacao) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.relatorio_energetico
					ADD FOREIGN KEY (id_energia)
					REFERENCES public.energia (id_energia) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;

				END;
		""")
		print("Tabelas criadas com sucesso.")
	except Exception as e:
		print(f"Erro ao criar tabelas: {e}")
	finally:
		close_pg_cursor(cursor)

## Funções para população individual dos bancos

In [ ]:
def populate_country_table(conn):
	cursor = conn.cursor()
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho

			for line in lines:
				# Pega os dados usando pandas
				country = line[0]
				country_code = line[1]
				continent = line[3]
				try:
					cursor.execute("""
						INSERT INTO pais (codigo, nome, continente)
						VALUES (%s, %s, %s)
						ON CONFLICT (codigo) DO NOTHING;
					""", (country_code, country, continent))		
				except Exception as e:
					print(f"Erro ao inserir país {country_code}: {e}")
					continue
		print("Tabela de país populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de país: {e}")
	finally:
		close_pg_cursor(cursor)

def populate_group_table(conn):
	cursor = conn.cursor()
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			id_grupo = 1
			header = next(lines)
			for grupo in header[5:10]:
				cursor.execute("""
					INSERT INTO grupo (id_grupo, sigla)
					VALUES (%s, %s)
					ON CONFLICT (id_grupo) DO NOTHING;
				""", (id_grupo, grupo))
				id_grupo += 1
		print("Tabela de grupo populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de grupo de {grupo}: {e}")
	finally:
		close_pg_cursor(cursor)

def populate_country_group_table(conn):
	cursor = conn.cursor()
	last_code = ''
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho
			for line in lines:
				if line[1] != last_code:
					last_code = line[1]
					for i in range(5, 10):
						print(line[i])
						if line[i] == '1.0':
							country_code = line[1]
							group_id = i - 4
							cursor.execute("""
								INSERT INTO pais_grupo (pais_codigo, grupo_id_grupo)
								VALUES (%s, %s);
							""", (country_code, group_id))
		print("Tabela país_grupo populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de país e grupo: {e}")
	finally:
		close_pg_cursor(cursor)

def populate_energy_interaction_table(conn):
	cursor = conn.cursor()
	added = []
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho
			id = 1
			for line in lines:
				# Pega os dados usando pandas
				category = line[10]
				unit = line[13]
				# Checa se a interação já existe
				if category not in added:
					added.append(category)
					try:
						cursor.execute("""
							INSERT INTO interacao_energetica (id_interacao, nome, unidade)
							VALUES (%s, %s, %s)
							ON CONFLICT (id_interacao) DO NOTHING;
						""", (id, category, unit))
						id += 1
					except Exception as e:
						print(f"Erro ao inserir interação {category}: {e}")
						continue
		print("Tabela de interação populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de interação: {e}")
	finally:
		close_pg_cursor(cursor)

def populate_energy_table(conn):
	cursor = conn.cursor()
	added = []
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho
			id = 1
			for line in lines:
				# Pega os dados usando pandas
				source = line[12]
				# Checa se a interação já existe
				if source not in added:
					added.append(source)
					try:
						cursor.execute("""
							INSERT INTO energia (id_energia, tipo)
							VALUES (%s, %s)
							ON CONFLICT (id_energia) DO NOTHING;
						""", (id, source))
						id += 1
					except Exception as e:
						print(f"Erro ao inserir interação {source}: {e}")
						continue
		print("Tabela de energia populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de energia: {e}")
	finally:
		close_pg_cursor(cursor)

def populate_energy_report_table(conn):
	cursor = conn.cursor()
	# cria um dicionario sources com id e nome das energias
	sources = {}
	try:
		cursor.execute("SELECT id_energia, tipo FROM energia;")
		rows = cursor.fetchall()
		for row in rows:
			sources[row[1]] = row[0]
	except Exception as e:
		print(f"Erro ao buscar fontes de energia: {e}")
		return
	# cria um dicionario categories com id e nome das interações
	categories = {}
	try:
		cursor.execute("SELECT id_interacao, nome FROM interacao_energetica;")
		rows = cursor.fetchall()
		for row in rows:
			categories[row[1]] = row[0]
	except Exception as e:
		print(f"Erro ao buscar categorias de energia: {e}")
		return
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho
			for line in lines:
				# Pega os dados usando pandas
				country_code = line[1]
				source = line[12]
				category = line[10]
				year = int(line[2])
				try:
					value = float(line[14])
					cursor.execute("""
						INSERT INTO relatorio_energetico (codigo_pais, id_interacao, id_energia, ano, valor)
						VALUES (%s, %s, %s, %s, %s);
					""", (country_code, categories[category], sources[source], year, value))
				except Exception as e:
					print(f"Erro ao inserir relatório energético: {e}")
					continue
		print("Tabela de relatório energético populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de relatório energético: {e}")
	finally:
		close_pg_cursor(cursor)

def populate_pib_table(conn):
    cursor = conn.cursor()
    try:
        with open(WORLD_DATA_CSV, 'r') as f:
            lines = csv.reader(f)
            next(lines)  # Pula o cabeçalho
            for line in lines:
                country_code = line[1]
                year = line[2]
                try:
                    value = float(line[5])
                    cursor.execute("""
                                    INSERT INTO pib (ano, valor_usd, codigo_pais)
                                    VALUES (%s, %s, %s);
                                    """, (year, value, country_code))
                except Exception as e:
                    continue
        print("Tabela de pib populada com sucesso.")
    except Exception as e:
        print(f"Erro ao popular tabela de pib: {e}")
    finally:
        close_pg_cursor(cursor)
        
def populate_gas_table(conn):
    cursor = conn.cursor()
    try:
        with open(WORLD_DATA_CSV, 'r') as f:
            lines = csv.reader(f)
            header = next(lines)
            names = []
            formulas = []
            carbon = header[3].split()
            names.append(" ".join([carbon[0], carbon[1]]))
            formulas.append(carbon[2])
            
            methane = header[6].split()
            names.append(methane[0])
            formulas.append(methane[1])
            
            nitrogen = header[7].split()
            names.append(" ".join([nitrogen[0], nitrogen[1]]))
            formulas.append(carbon[2])
            
            greenhouse = header[12].split()
            names.append(" ".join([greenhouse[0], greenhouse[1]]))
            formulas.append("")
            for i in range(4):
                try:
                    cursor.execute("""
                                INSERT INTO gas (id_gas, nome_gas, formula_gas)
                                VALUES (%s, %s, %s)
                                ON CONFLICT (id_gas) DO NOTHING;
                                """, (i+1, names[i], formulas[i]))
                except Exception as e:
                    print(f"Erro ao inserir gas {names[i]}: {e}")
                    continue
            print("Tabela gas populada com sucesso.")            
    except Exception as e:
        print(f"Erro ao popular tabela de gases: {e}")
    finally:
        close_pg_cursor(cursor)

def populate_gas_emission_table(conn):
    cursor = conn.cursor()
    try:
        with open(WORLD_DATA_CSV, 'r') as f:
            lines = csv.reader(f)
            gases_columns = [3, 6, 7, 12]
            next(lines)
            for line in lines:
                country_code = line[1]
                year = line[2]
                try:
                    for id in range(4):
                        gas = float(line[gases_columns[id]])
                        cursor.execute("""
                                    INSERT INTO emissao_gas (gas_id_gas, pais_codigo, ano, emissao_mtco2)
                                    VALUES (%s, %s, %s, %s)
                                    """, (id+1, country_code, year, gas))
                except Exception as e:
                    print(f"Erro ao inserir gas_emissao {id+1}, {year}: {e}")
                    continue
            print("Tabela gas populada com sucesso.")            
    except Exception as e:
        print(f"Erro ao popular tabela de gases: {e}")
    finally:
        close_pg_cursor(cursor)

def populate_area_table(conn):
    cursor = conn.cursor()
    try:
        with open(WORLD_DATA_CSV, 'r') as f:
            lines = csv.reader(f)
            header = next(lines)
            area_columns = [9, 11, 13, 4]
            try:
                for id in range(4):
                    tipo = header[area_columns[id]].split()[0]
                    cursor.execute("""
                                   INSERT INTO area (id, tipo)
                                   VALUES (%s, %s)
                                   ON CONFLICT (id) DO NOTHING;
                                   """, (id+1, tipo))
            except Exception as e:
                print(f"Erro ao inserir area {tipo}: {e}")
            print("Tabela areas populada com sucesso.")
    except Exception as e:
        print(f"Erro ao popular tabela de areas: {e}")
    finally:
        close_pg_cursor(cursor)
    
def populate_demography_table(conn):
    cursor = conn.cursor()
    try:
        with open(WORLD_DATA_CSV, 'r') as f:
            lines = csv.reader(f)
            next(lines)
            population_columns = [10, 8, 14]
            for line in lines:
                country_code = line[1]
                year = line[2]
                try:
                    for id in range(3):
                        value = int(line[population_columns[id]][:-2])
                        cursor.execute("""
                                    INSERT INTO demografia (area_id, pais_codigo, ano, num_habitantes)
                                    VALUES (%s, %s, %s, %s)
                                    """, (id+1, country_code, year, value))
                except Exception as e:
                    print(f"Erro ao inserir numero de habitantes {id+1}, {year}, {country_code}: {e}")
                    continue
            print("Tabela demografia populada com sucesso.")   
    except Exception as e:
        print(f"Erro ao popular tabela de demografia: {e}")
    finally:
        close_pg_cursor(cursor)

def populate_territorial_distribution_table(conn):
    cursor = conn.cursor()
    try:
        with open(WORLD_DATA_CSV, 'r') as f:
            lines = csv.reader(f)
            next(lines)
            area_columns = [9, 11, 13, 4]
            for line in lines:
                country_code = line[1]
                year = line[2]
                try:
                    for id in range(4):
                        value = float(line[area_columns[id]])
                        cursor.execute("""
                                    INSERT INTO distribuicao_territorial (pais_codigo, area_id, ano, area_km2)
                                    VALUES (%s, %s, %s, %s);
                                    """, (country_code, id+1, year, value))
                except Exception as e:
                    print(f"Erro ao inserir area {id+1}, {year}, {country_code}: {e}")
                    continue
            print("Tabela distribuição territorial populada com sucesso.")   
    except Exception as e:
        print(f"Erro ao popular tabela de distribuição territorial: {e}")
    finally:
        close_pg_cursor(cursor)

# Main block

In [ ]:
conn = create_pg_connection()
# Se for rodar pela primeira vez, torne todas as variáveis True
populate = False
populate_country = False
populate_group = False
populate_country_group = False
populate_interaction = False
populate_energy = False
populate_energy_report = False
populate_pib = True
populate_gas = False
populate_gas_emission = False
populate_area = False
populate_demography = False
populate_territorial_distribution = False

if conn:
		create_tables(conn)
		if populate:
			if PROC_ENERGY_CSV:
				if populate_country:
					populate_country_table(conn)
				if populate_group:
					populate_group_table(conn)
				if populate_country_group:
					populate_country_group_table(conn)
				if populate_interaction:
					populate_energy_interaction_table(conn)
				if populate_energy:
					populate_energy_table(conn)
				if populate_energy_report:
					populate_energy_report_table(conn)
			else:
				print(f"Arquivo {PROC_ENERGY_CSV} não encontrado.")
			if WORLD_DATA_CSV:
				if populate_pib:
					populate_pib_table(conn)
				if populate_gas:
					populate_gas_table(conn)
				if populate_gas_emission:
					populate_gas_emission_table(conn)
				if populate_area:
					populate_area_table(conn)
				if populate_demography:
					populate_demography_table(conn)
				if populate_territorial_distribution:
					populate_territorial_distribution_table(conn)
			else:
				print(f"Arquivo {WORLD_DATA_CSV} não encontrado.")
		end_pg_connection(conn)

# Query 1: Porcentagem de energia renovável produzida por grupo econômico e ano
Detecta a porcentagem renovável da energia total gerada por grupo econômico a cada ano.

In [ ]:
query = """
SELECT RE.ANO,
		G.SIGLA,
		ROUND((SUM(RE.VALOR)*100 / T.TOTAL)::NUMERIC, 2) AS PORCENTAGEM_RENOVAVEL
FROM PAIS P
INNER JOIN PAIS_GRUPO PG
	ON P.CODIGO = PG.PAIS_CODIGO
INNER JOIN GRUPO G
	ON PG.GRUPO_ID_GRUPO = G.ID_GRUPO
INNER JOIN RELATORIO_ENERGETICO RE
	ON P.CODIGO = RE.CODIGO_PAIS
INNER JOIN INTERACAO_ENERGETICA IE
	ON RE.ID_INTERACAO = IE.ID_INTERACAO
INNER JOIN ENERGIA E
	ON RE.ID_ENERGIA = E.ID_ENERGIA
INNER JOIN (
	SELECT	RE.ANO,
			G.SIGLA,
			SUM(RE.VALOR) AS TOTAL
		FROM PAIS P
	INNER JOIN PAIS_GRUPO PG
		ON P.CODIGO = PG.PAIS_CODIGO
	INNER JOIN GRUPO G
		ON PG.GRUPO_ID_GRUPO = G.ID_GRUPO
	INNER JOIN RELATORIO_ENERGETICO RE
		ON P.CODIGO = RE.CODIGO_PAIS
	INNER JOIN INTERACAO_ENERGETICA IE
		ON RE.ID_INTERACAO = IE.ID_INTERACAO
	INNER JOIN ENERGIA E
		ON RE.ID_ENERGIA = E.ID_ENERGIA
	WHERE IE.NOME LIKE '%generation'
	GROUP BY RE.ANO, G.SIGLA
) T
	ON T.ANO = RE.ANO AND T.SIGLA = G.SIGLA 
WHERE E.TIPO != 'Coal'
	AND E.TIPO != 'Gas'
	AND E.TIPO != 'Nuclear'
	AND E.TIPO != 'Other Fossil'
	AND IE.NOME LIKE '%generation'
GROUP BY RE.ANO, G.SIGLA, IE.NOME, T.TOTAL, IE.UNIDADE
ORDER BY RE.ANO, G.SIGLA;
"""
conn = create_pg_connection()
execute_query(conn, query, 1)
end_pg_connection(conn)

# Query 2: PIB por energia (per capita)
Compara os valores de Produto Interno Bruto per capita e energia per capita por país.

In [ ]:
query = """
SELECT 
    p.nome AS pais,
    re.ano AS ano,
	ROUND(SUM(CASE WHEN ie.nome = 'Electricity generation' THEN re.valor ELSE 0 END)::numeric * 1000000000 / d.num_habitantes, 4) AS energia_produzida_per_capita_kWh,
    ROUND(SUM(CASE WHEN ie.nome = 'Electricity demand' THEN re.valor ELSE 0 END)::numeric * 1000000000 / d.num_habitantes, 4) AS energia_consumida_per_capita_kWh,
    ROUND(pib.valor_usd::numeric / d.num_habitantes, 2) AS pib_per_capita
FROM 
    relatorio_energetico re
INNER JOIN 
    pais p ON re.codigo_pais = p.codigo
INNER JOIN 
    interacao_energetica ie ON re.id_interacao = ie.id_interacao
INNER JOIN 
    energia e ON re.id_energia = e.id_energia
INNER JOIN
    pib ON pib.codigo_pais = re.codigo_pais AND pib.ano = re.ano
INNER JOIN
    demografia d ON d.pais_codigo = re.codigo_pais AND d.ano = re.ano
INNER JOIN
 area a ON a.id = d.area_id
WHERE  
    ie.nome IN ('Electricity demand', 'Electricity generation') AND a.tipo = 'Surface'
GROUP BY 
    p.nome, re.ano, ie.unidade, d.num_habitantes, pib.valor_usd
ORDER BY 
    p.nome, re.ano;
"""
conn = create_pg_connection()
execute_query(conn, query, 2)
end_pg_connection(conn)

# Query 3: Relação entre matriz energética e emissão de co2 per capita, por país e ano
Compara os valores, buscando encontrar relação entre geração de energias não renováveis e emissão de co2. O critério de apresentação dos dados é por ordem decrescente de emissão per capita de co2.

In [ ]:
query = """
WITH producao_total AS (
  SELECT 
    r2.codigo_pais,
    r2.ano,
    SUM(r2.valor)::numeric AS valor_total
  FROM relatorio_energetico r2
  INNER JOIN interacao_energetica ie2 ON r2.id_interacao = ie2.id_interacao
  WHERE ie2.nome = 'Electricity generation'
    AND r2.ano = 2010
  GROUP BY r2.codigo_pais, r2.ano
),
relatorio_filtrado AS (
  SELECT 
    re.codigo_pais,
    re.ano,
    re.valor,
    ie.unidade,
    e.tipo
  FROM relatorio_energetico re
  INNER JOIN interacao_energetica ie ON re.id_interacao = ie.id_interacao
  INNER JOIN energia e ON re.id_energia = e.id_energia
  WHERE ie.nome = 'Electricity generation'
    AND re.ano = 2010
),
emissao_filtrada AS (
  SELECT 
    eg.pais_codigo,
    eg.ano,
    eg.emissao_mtco2
  FROM emissao_gas eg
  INNER JOIN gas g ON g.id_gas = eg.gas_id_gas
  WHERE g.nome_gas = 'Carbon dioxide' AND eg.ano = 2010
),
demografia_filtrada AS (
  SELECT 
    d.pais_codigo,
    d.ano,
    d.num_habitantes
  FROM demografia d
  INNER JOIN area a ON a.id = d.area_id
  WHERE a.tipo = 'Surface' AND d.ano = 2010
)

SELECT 
  p.nome AS pais,
  rf.ano,
  rf.tipo AS tipo_energia,
  ROUND(CAST(rf.valor * 100.0 / pt.valor_total AS NUMERIC), 2) AS producao_energia_porcentagem,
  ROUND(CAST(1000000 * ef.emissao_mtco2 * 100.0 / df.num_habitantes AS NUMERIC), 4) AS emissao_co2_per_capita
FROM relatorio_filtrado rf
INNER JOIN pais p ON p.codigo = rf.codigo_pais
INNER JOIN producao_total pt ON pt.codigo_pais = rf.codigo_pais AND pt.ano = rf.ano
INNER JOIN demografia_filtrada df ON df.pais_codigo = rf.codigo_pais AND df.ano = rf.ano
INNER JOIN emissao_filtrada ef ON ef.pais_codigo = rf.codigo_pais AND ef.ano = rf.ano
WHERE rf.valor > 0
ORDER BY emissao_co2_per_capita DESC, p.nome ASC, producao_energia_porcentagem DESC;
"""
conn = create_pg_connection()
execute_query(conn, query, 3)
end_pg_connection(conn)

# Query 4: Energia gerada por km² de área florestal
Traça relação entre geração de energia e área florestal no ano de 2015.

In [ ]:
query = """
SELECT
    p.nome AS nome_pais,
    re.ano,
    ROUND(CAST(1000000*SUM(re.valor) AS NUMERIC), 0) AS energia_total_Mwh,
    ROUND(CAST(dt.area_km2 AS NUMERIC), 0) AS area_florestal_km2,
    ROUND(CAST(1000000*SUM(re.valor) / NULLIF(dt.area_km2, 0) AS NUMERIC),2) AS Mwh_por_km2_florestal
FROM
    pais p
INNER JOIN
    relatorio_energetico re ON re.codigo_pais = p.codigo
INNER JOIN
    distribuicao_territorial dt ON dt.pais_codigo = p.codigo AND dt.ano = re.ano
INNER JOIN
    energia e ON e.id_energia = re.id_energia
INNER JOIN
    interacao_energetica ie ON ie.id_interacao = re.id_interacao
INNER JOIN
    area a ON a.id = dt.area_id
WHERE
    a.tipo = 'Forest' AND ie.nome = 'Electricity generation'
	AND RE.ANO = 2015
GROUP BY
    p.nome, re.ano, dt.area_km2
ORDER BY
    p.nome, re.ano;
"""	
conn = create_pg_connection()
execute_query(conn, query, 4)
end_pg_connection(conn)

# Query 5: Razão entre habitantes rurais e urbanos por razão entre energia renovável e não renovável
Compara as taxas para valores não nulos.

In [ ]:
query = """
SELECT DISTINCT ON (ANO, NOME) *
FROM (
	SELECT D1.ANO,
			P.NOME,
			ROUND((CAST(D1.NUM_HABITANTES AS FLOAT) / NULLIF(D2.NUM_HABITANTES, 0))::NUMERIC, 2) AS RAZAO_POP_RURAL_URBANA,
			ROUND((CAST(RE1.VALOR AS FLOAT) / NULLIF(RE2.VALOR, 0))::NUMERIC, 2) AS RAZAO_ENERGIA_RENOV_NAO_RENOV
	FROM DEMOGRAFIA D1
	INNER JOIN DEMOGRAFIA D2
		ON D1.ANO = D2.ANO
			AND D1.PAIS_CODIGO = D2.PAIS_CODIGO
	INNER JOIN RELATORIO_ENERGETICO RE1
		ON D1.ANO = RE1.ANO
			AND D1.PAIS_CODIGO = RE1.CODIGO_PAIS
	INNER JOIN RELATORIO_ENERGETICO RE2
		ON D1.ANO = RE2.ANO
			AND D1.PAIS_CODIGO = RE2.CODIGO_PAIS
	INNER JOIN ENERGIA E1
		ON RE1.ID_ENERGIA = E1.ID_ENERGIA
	INNER JOIN ENERGIA E2
		ON RE2.ID_ENERGIA = E2.ID_ENERGIA
	INNER JOIN INTERACAO_ENERGETICA IE1
		ON RE1.ID_INTERACAO = IE1.ID_INTERACAO
	INNER JOIN INTERACAO_ENERGETICA IE2
		ON RE2.ID_INTERACAO = IE2.ID_INTERACAO
	INNER JOIN PAIS P
		ON D1.PAIS_CODIGO = P.CODIGO
	WHERE D1.AREA_ID = 1
		AND D2.AREA_ID = 3
		AND E1.TIPO != 'Coal'
		AND E1.TIPO != 'Gas'
		AND E1.TIPO != 'Nuclear'
		AND E1.TIPO != 'Other Fossil'
		AND (E2.TIPO = 'Coal'
		OR E2.TIPO = 'Gas'
		OR E2.TIPO = 'Nuclear'
		OR E2.TIPO = 'Other Fossil')
		AND IE1.NOME LIKE '%generation'
		AND IE2.NOME LIKE '%generation'	
	GROUP BY D1.PAIS_CODIGO, D1.ANO, D1.NUM_HABITANTES, D2.NUM_HABITANTES, RE1.VALOR, RE2.VALOR, P.NOME
	ORDER BY D1.ANO, P.NOME
) T
WHERE RAZAO_POP_RURAL_URBANA IS NOT NULL
	AND RAZAO_ENERGIA_RENOV_NAO_RENOV IS NOT NULL
	AND RAZAO_ENERGIA_RENOV_NAO_RENOV != 0;
"""	
conn = create_pg_connection()
execute_query(conn, query, 5)
end_pg_connection(conn)